In [1]:
import os
import yaml
import tifffile
import numpy as np

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
# from src import measures
from src.cleaner import DoubleStepCleaner

In [2]:
fpath = "/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/"
db_path = "/home/ws/er5241/Repos/measuring-repo/artifacts/debug.json"

fls = os.listdir(fpath)
# id = np.random.choice(fls)

In [3]:
def initialize(id, main_organ):
  img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
  msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"
  img = tifffile.imread(img_path)
  msk = tifffile.imread(msk_path)

  cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"
  with open(cfg_path, "r") as stream:
      configs = yaml.safe_load(stream)

  cleaning_config = configs['cleaning']
  measuring_config = configs['measures']
  centering_config = configs.get('centering', [])

  cleaner = DoubleStepCleaner(**cleaning_config)
  msk_clean, roi = cleaner(msk)
  img_clean = img[roi]

  centers = {}
  for centering in centering_config:
      centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

  return img_clean, msk_clean, measuring_config, centers

#### Original approach

In [4]:
def get_bootstrapped_radii(markup, samples=10, size=5000):
  point_cloud = np.stack(np.where(markup), 1)

  all_radii = []
  for i in range(samples):
      eltool = EllipsoidTool()
      ell = eltool.getMinVolEllipse(point_cloud[np.random.choice(len(point_cloud), size, replace=False)])
      all_radii.append(np.sort(ell[1]))
  all_radii = np.stack(all_radii, 0)

  median_radii = np.median(all_radii, 0)
  return median_radii

#### Convex Hull approach

In [5]:
from scipy.spatial import ConvexHull
from src.ellipsoid_tool import EllipsoidTool

def get_radii(markup):
  mp = np.dstack(np.where(markup))[0] # marked points
  ch = ConvexHull(mp)
  hp = ch.points[ch.vertices]  # hull vertice coordinates

  eltool = EllipsoidTool()
  ell = eltool.getMinVolEllipse(hp)
  radii = np.sort(ell[1])
  
  return radii

In [6]:
def eccentricity_meridional(markup, volume, type_):
    """Calculates meridional eccentricity of the circumscribed ellipsoid of organ"""
    r1, r2, r3 = get_radii(markup) if type_=='hull' else get_bootstrapped_radii(markup)
    return (r3 - r1) / (r3 + r1)

def eccentricity_equatorial(markup, volume, type_):
    """Calculates equatorial eccentricity of the circumscribed ellipsoid of organ"""
    r1, r2, r3 = get_radii(markup) if type_=='hull' else get_bootstrapped_radii(markup)
    return (r3 - r2) / (r3 + r2)

In [7]:
def compare_measurements():
  for lbl in measuring_config:
    try:
      lbl_mask = (msk_clean == lbl['id'])
      print('  ',lbl['id'], lbl['name'])

      print('    eccentricity_meridional')
      measurement_ch = eccentricity_meridional(lbl_mask, img_clean, type_='hull')
      print('      convex hull', measurement_ch)
      measurement = eccentricity_meridional(lbl_mask, img_clean, type_='original')
      print('      original', measurement)
      diff = abs(measurement_ch - measurement)
      print('      difference',diff)
      differences.append(diff)

      print('    eccentricity_equatorial')
      measurement_ch = eccentricity_equatorial(lbl_mask, img_clean, type_='hull')
      print('      convex hull', measurement_ch)
      measurement = eccentricity_equatorial(lbl_mask, img_clean, type_='original')
      print('      original', measurement)
      diff = abs(measurement_ch - measurement)
      print('      difference',abs(measurement_ch - measurement))
      differences.append(diff)
    except:
      print('      original failed')

    print()
  return

In [8]:
samples = ['Medaka_1115_20-1', 'Medaka_1119_23-2', 'Medaka_1129_25-1']
# samples = np.random.choice(fls,5)

differences = []

for sample in samples:
  print(sample)
  img_clean, msk_clean, measuring_config, centers = initialize(id=sample, main_organ='heartkidney')
  compare_measurements()
  print()

Medaka_1115_20-1
   2 atrium
    eccentricity_meridional
      convex hull 0.4751972550823252
      original failed

   4 left_kidney
    eccentricity_meridional
      convex hull 0.5204702555380479
      original 0.5210178396488037
      difference 0.0005475841107557278
    eccentricity_equatorial
      convex hull 0.3171896165216324
      original 0.34038017576694896
      difference 0.023190559245316544

   5 ventricle
    eccentricity_meridional
      convex hull 0.33688678244200115
      original 0.3346072778149912
      difference 0.0022795046270099317
    eccentricity_equatorial
      convex hull 0.2910228974260339
      original 0.301916146331551
      difference 0.010893248905517128

   6 right_kidney
    eccentricity_meridional
      convex hull 0.5484640723206083
      original 0.5448536183634654
      difference 0.0036104539571429894
    eccentricity_equatorial
      convex hull 0.44013885415765447
      original 0.4467079111321183
      difference 0.006569056974463805


Me

In [9]:
print('min',np.min(differences))
print('mean',np.mean(differences))
print('max',np.max(differences))

min 0.00019007622043210048
mean 0.006528799304649366
max 0.023190559245316544
